# Run AI reviewer for application


In [2]:
# This is test code for comparing logic. I compared two methods for the task. 
# In the first baseline method, the AI (gpt-4o-2024-05-13) accepts the full texts of the L/C application and the contract, then identifies any discrepancies. 
# In the second method, the AI first splits the L/C application into parts and then checks each part for discrepancies against the contract.
# The second method is used in the code.

import os
from dotenv import load_dotenv
load_dotenv()  # Load variables from .env

from llm import create_client, get_response_from_llm
from review import extract_pdf_text

# Create a real client
api_key = os.getenv("OPENAI_API_KEY")
client, model = create_client("gpt-4o-2024-05-13")

file_path = "docs/application/002" # Sample with mistakes in L/C application

it_reviewer_system_prompt = (
    "You are an AI reviewer in the international trading company responsible for evaluating Letter of Credit (L/C) applications. "
    "Your mission is to assess if all contents described in L/C application are align with the contract. "
    "Be critical and cautious in your decision-making process. Your evaluations should be structured, concise, and accurate."
)

Using OpenAI API with model gpt-4o-2024-05-13.


## Method 1: Simple Comparison


In [ ]:
contract_text = extract_pdf_text(file_path + "/contract.pdf")
application_text = extract_pdf_text(file_path + "/application.pdf")

base_prompt = f"""
Below is a L/C application. Please carefully read the content and advice if any discrepancy exists with given contract.
Please only provide identified discrepancy in your response.
If you do not find any discrepancy, just mention no discrepancy.

L/C APPLICATION:
{application_text}

PROVIDED CONTRACT:
{contract_text}
"""

llm_review, msg_history = get_response_from_llm(
    base_prompt,
    model=model,
    client=client,
    system_message=it_reviewer_system_prompt,
    print_debug=False,
)

print(llm_review)

## Method 2: Split, Compare and Summarize


In [ ]:
import re
import json

base_prompt = """
Please split following document into multiple parts by content.
*Please just split the original text without changing any words even if original text contains typo or mistake.
*Please split without overlapping
*Please provide output in JSON.

Args: 
    application text (str): A texts from document 

Returns:
    dict: A list of splitted text.

    output format should be as below:
    * Please only provide JSON output as string starting from { and ending with }

    {
        "output": ["list of splitted texts", "list of splitted texts", "list of splitted texts"...]
        ...
    }

"""

application_text = extract_pdf_text(file_path + "/application.pdf")
base_prompt += f"Document: {application_text}"

llm_review, msg_history = get_response_from_llm(
    base_prompt,
    model=model,
    client=client,
    system_message=it_reviewer_system_prompt,
    print_debug=False,
)

try:
    json_match = re.search(r"{.*}", llm_review, re.DOTALL)
    if json_match:
        cleaned_review = json_match.group(0)
        review = json.loads(cleaned_review)
    else:
        raise ValueError("No JSON found in response")
except json.JSONDecodeError as e:
    print(f"JSON decoding failed: {e}")
    print(f"LLM response: {llm_review}")
    review = None

print(review)

In [ ]:
contract_text = extract_pdf_text(file_path + "/contract.pdf")
result = ""
for text in review["output"]:
    base_prompt = f"""
    Below paragraph is extracted from a L/C application. Please carefully read the content and advice if any discrepancy exists with given contract.
    Please only provide identified discrepancy in your response.
    If you do not find any discrepancy, just mention no discrepancy.

    EXTRACTED PARAGRAPH FROM L/C APPLICATION:
    {text}

    PROVIDED CONTRACT:
    {contract_text}
    """

    llm_review, msg_history = get_response_from_llm(
        base_prompt,
        model=model,
        client=client,
        system_message=it_reviewer_system_prompt,
        print_debug=False,
    )

    result += llm_review

print(result)
    

In [ ]:
base_prompt = f"""
    Please summarize the discrepancy found in the the below report.

    Report:
    {result}
    """

llm_review, msg_history = get_response_from_llm(
    base_prompt,
    model=model,
    client=client,
    system_message=it_reviewer_system_prompt,
    print_debug=False,
)

print(llm_review)
